In [2]:
import random
import pandas as pd
from io import StringIO
import re
import yaml
import xgboost
import joblib
import numpy as np
import pandas as pd
players = pd.read_csv('C:/Github/nfl_sim/data/depth.csv',
                      usecols=['season', 'club_code', 'week',  'depth_team','formation', 'gsis_id',
       'position', 'full_name'])

In [3]:
class QB:
    def __init__(self, d):  # noqa: F811
        self.name = d["full_name"]
        self.attempts = 0
        self.completions = 0
        self.rush_yards = 0
        self.pass_yards = 0
        self.td = 0
        self.int = 0

    def __repr__(self):
        return f"QB:{self.name} has {self.completions} completions for {self.pass_yards} yards"


class RB:
    def __init__(self, d):
        self.name = d["full_name"]
        self.rush_yards = 0
        self.receiving_yards = 0
        self.targets = 0
        self.receptions = 0
        self.td = 0
        self.carries = 0

    def __repr__(self):
        return f"RB:{self.name} has {self.carries} carries for {self.rush_yards} yards"


class WR:
    def __init__(self, d):
        self.name = d["full_name"]
        self.rush_yards = 0
        self.receiving_yards = 0
        self.targets = 0
        self.receptions = 0
        self.td = 0
        self.carries = 0

    def __repr__(self):
        return f"WR:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class Team:
    def __init__(self, depth_chart: pd.DataFrame):
        self.name = depth_chart["team"].iloc[0]
        self.QB = QB(depth_chart)
        self.RB = RB(depth_chart)
        self.WR = WR(depth_chart)
        self.score = 0
        self.plays = 0


class Team2:
    def __init__(self, name, season, week, use_current_injuries=False):
        self.name = name
        self.score = 0
        self.plays = 0
        self.players = players.loc[
            (players.club_code == name) & (players.season == season)
        ]
        self.players = self.players.loc[
            (self.players.week == min(self.players.week.max(), week))
            & (self.players.formation == "Offense")
            & (self.players.position.isin(["QB", "WR", "TE", "RB"]))
        ]

        self.QBs = self.get_players_by_position("QB")
        self.RBs = self.get_players_by_position("RB")
        self.WRs = self.get_players_by_position("WR")

    def get_players_by_position(self, position):
        """Filter players by position and create player objects."""
        position_data = self.players[(self.players["position"] == position)]

        # Create player objects based on position
        players = []
        for _, player_data in position_data.iterrows():
            if position == "WR":
                players.append(WR(player_data))
            elif position == "RB":
                players.append(RB(player_data))
            elif position == "QB":
                players.append(QB(player_data))
        return players

In [4]:
vik = Team2('PHI',2024, 13)
det = Team2('DET',2024, 13)
vik.players.sample(5)

,season,club_code,week,depth_team,formation,gsis_id,position,full_name
528745,2024,PHI,12.0,2,Offense,00-0038102,QB,Kenny Pickett
528722,2024,PHI,12.0,2,Offense,00-0036919,RB,Kenneth Gainwell
528746,2024,PHI,12.0,3,Offense,00-0038400,QB,Tanner McKee
528732,2024,PHI,12.0,1,Offense,00-0035676,WR,A.J. Brown
528721,2024,PHI,12.0,2,Offense,00-0037086,TE,Grant Calcaterra


In [5]:
det.RBs

[RB:Jahmyr Gibbs has 0 carries for 0 yards,
 RB:David Montgomery has 0 carries for 0 yards,
 RB:Craig Reynolds has 0 carries for 0 yards,
 RB:Sione Vaki has 0 carries for 0 yards]

In [6]:
class GameState:


	def __init__(self, home, away, config):
		self.home =home
		self.away = away
		self.quarter = 1
		self.possession = None
		self.down = 1
		self.togo = 10
		self.ball_position = 35  # Yardline (0-100, 50 is midfield)
		self.clock = 900  # Seconds in the current quarter (15 mins = 900 seconds)
		self.home_timeouts = 3
		self.away_timeouts = 3
		self.pbp = []
		self.run_or_pass = joblib.load('models/run_or_pass.joblib')
		self.run_or_pass_cols = config['run_or_pass_cols']
		self.play_encoding = config['play_encoding']
		self.wind = random.randint(0,10)
		self.temp = random.randint(40,90)
		self.play_functions = {
			'field_goal': self.field_goal,
			'no_play': self.run_play,
			'pass': self.pass_play,
			'punt': self.punt,
			'qb_kneel': self.qb_kneel,
			'qb_spike': self.qb_spike,
			'run': self.run_play
			}
		
		
	def switch_poss(self):
		self.possession = self.away if self.possession == self.home else self.home 
		self.ball_position = 100 - min(self.ball_position, 100)
		self.down = 1
		self.togo = min(10, (100-self.ball_position))
		return
	
	def kickoff(self):
		self.switch_poss()
		self.ball_position = 35
		pass
	
	def start_game(self):
		lost_kickoff = random.choice((self.home, self.away))
		self.possession=lost_kickoff
		self.kickoff()
		print(f'{self.possession.name} has won the kickoff')
		self.log_play()
		self.log_play_type('kickoff',0)

	def play(self, team):
		self.log_play()
		team.plays += 1
		features = [self.pbp[-1][key] for key in self.run_or_pass_cols]
		preds = self.run_or_pass.predict_proba([features])
		play_type_int = np.random.choice(len(preds[0]), p=preds[0])
		play_type = self.play_encoding.get(play_type_int, 1)
		yds = self.play_functions[play_type](team)
		self.togo -= yds
		self.ball_position += yds
		self.clock -= random.randint(10, 40)
		self.td_check(team)
		self.check_downs(team)
		self.log_play_type(play_type, yds)


	def log_play(self):
		"""Logs the context of the game state at each play."""
		play_data = {
			"possession": self.possession.name,
			"quarter": self.quarter,
			"down": self.down,
			"ydstogo": self.togo,
			"goal_to_go": int(self.ball_position >= 100-self.togo),
			"yardline_100": self.ball_position,
			"total_home_score": self.home.score,
			"total_away_score": self.away.score,
			"posteam_score": self.possession.score,
			"score_differential": (self.home.score - self.away.score),
			"wind": self.wind,
			"temp": self.temp,
			"quarter_seconds_remaining": self.clock,
			'half_seconds_remaining': self.clock + (900 * (self.quarter%2)),
    		'game_seconds_remaining': self.clock + (900 * (4-self.quarter)),
			'spread_line':-14.5,
			'total_line': 42.5
		}

		self.pbp.append(play_data)

	def log_play_type(self,play_type, yds):
		self.pbp[-1]['play_type'] = play_type
		self.pbp[-1]['yards_gained'] = yds
		print(
			f'{self.possession.name} {play_type} for {yds} yards, {self.pbp[-1]['yardline_100']} yd line,' + 
			  f' {self.pbp[-1]['ydstogo']} yds to go on {self.pbp[-1]['down']} down.' + 
			f' {self.pbp[-1]['quarter_seconds_remaining'] // 60}:{self.pbp[-1]['quarter_seconds_remaining']  % 60} left'
		)
		return

	def run_play(self, team):
		player = random.choice(team.RBs)
		player.carries += 1
		yds = random.randint(-5, 15)
		player.rush_yards += yds
		return yds

	def pass_play(self, team):
		passer = team.QBs[0]
		receiver = random.choice(team.WRs)
		passer.attempts += 1
		receiver.targets += 1
		if random.randint(0, 100) > 50:
			passer.completions += 1
			receiver.receptions += 1
			yds = random.randint(-1, 30)
			receiver.receiving_yards += yds
			passer.pass_yards += yds
		else:
			yds = 0
		return yds
	
	
	def punt(self, team):
		self.switch_poss()
		self.ball_position -= random.randint(30, 60)
		if self.ball_position < 0:
			self.ball_position = 25
		return 0
	
	def field_goal(self,team):
		result = random.randint(0,100)
		if result > 50:
			team.score += 3
			self.switch_poss()
			self.ball_position = 35
			print(f'{team.name} scored a FG')
		else:
			print(f'{team.name} missed FG')
			self.switch_poss()
		return 0
	
	def qb_kneel(self, team):
		# Implementation of qb kneel play
		print("QB kneel executed.")
		return -1

	def qb_spike(self, team):
		# Implementation of qb spike play
		print("QB spike executed.")
		return 0
	
	def td_check(self, team):
		if self.ball_position > 100:
			team.score += 7
			self.kickoff()
			print(f'{team.name} scored a TD')
		return
	
	def check_downs(self, team):
		if self.togo <= 0:
			self.togo = 10
			self.down = 1
		elif self.down == 4:
			self.switch_poss()
		else:
			self.down += 1


	def play_quarter(self):
		self.clock = 900
		while self.clock > 0:
			self.play(self.possession)
		self.quarter += 1
		print(f'{self.home.name}:{self.home.score}')
		print(f'{self.away.name}:{self.away.score}')

	def play_game(self):
		while self.quarter <= 4:
			self.play_quarter()
			print(self.quarter)


In [7]:
vik.QBs

[QB:Jalen Hurts has 0 completions for 0 yards,
 QB:Kenny Pickett has 0 completions for 0 yards,
 QB:Tanner McKee has 0 completions for 0 yards]

In [8]:
with open('models/feature_config.yaml', 'r') as file:
    config = yaml.safe_load(file)
game = GameState(vik, det, config)
game.start_game()
game.play_game()


DET has won the kickoff
DET kickoff for 0 yards, 35 yd line, 10 yds to go on 1 down. 15:0 left
DET run for 10 yards, 35 yd line, 10 yds to go on 1 down. 15:0 left
DET pass for 15 yards, 45 yd line, 10 yds to go on 1 down. 14:41 left
DET pass for 0 yards, 60 yd line, 10 yds to go on 1 down. 14:11 left
DET run for 12 yards, 60 yd line, 10 yds to go on 2 down. 13:35 left
DET run for -1 yards, 72 yd line, 10 yds to go on 1 down. 13:14 left
DET pass for 0 yards, 71 yd line, 11 yds to go on 2 down. 12:47 left
DET pass for 21 yards, 71 yd line, 11 yds to go on 3 down. 12:36 left
DET pass for 0 yards, 92 yd line, 10 yds to go on 1 down. 12:0 left
DET run for 8 yards, 92 yd line, 10 yds to go on 2 down. 11:31 left
DET pass for 0 yards, 100 yd line, 2 yds to go on 3 down. 11:1 left
PHI punt for 0 yards, 100 yd line, 2 yds to go on 4 down. 10:22 left
PHI pass for 0 yards, 25 yd line, 10 yds to go on 2 down. 10:2 left
PHI pass for 7 yards, 25 yd line, 10 yds to go on 3 down. 9:52 left
PHI scored a

In [9]:
print(game.away.plays, game.home.plays)

71 71


In [10]:
pbp = pd.json_normalize(game.pbp)
pbp

,possession,quarter,down,ydstogo,goal_to_go,yardline_100,total_home_score,total_away_score,posteam_score,score_differential,wind,temp,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,spread_line,total_line,play_type,yards_gained
0,DET,1,1,10,0,35,0,0,0,0,1,87,900,1800,3600,-14.5,42.5,kickoff,0
1,DET,1,1,10,0,35,0,0,0,0,1,87,900,1800,3600,-14.5,42.5,run,10
2,DET,1,1,10,0,45,0,0,0,0,1,87,881,1781,3581,-14.5,42.5,pass,15
3,DET,1,1,10,0,60,0,0,0,0,1,87,851,1751,3551,-14.5,42.5,pass,0
4,DET,1,2,10,0,60,0,0,0,0,1,87,815,1715,3515,-14.5,42.5,run,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,PHI,4,4,10,0,64,13,20,13,-7,1,87,98,98,98,-14.5,42.5,punt,0
139,DET,4,2,10,0,25,13,20,20,-7,1,87,68,68,68,-14.5,42.5,pass,13
140,DET,4,1,10,0,38,13,20,20,-7,1,87,56,56,56,-14.5,42.5,no_play,-1
141,DET,4,2,11,0,37,13,20,20,-7,1,87,25,25,25,-14.5,42.5,pass,0


In [11]:
features = [game.pbp[-5][key] for key in game.run_or_pass_cols]
preds = game.run_or_pass.predict_proba([features])
features

[64, 4, 0, 10, 13, -7, 98, 98, 98, 1, 87, -14.5, 42.5]

In [12]:
game.run_or_pass_cols

['yardline_100',
 'down',
 'goal_to_go',
 'ydstogo',
 'posteam_score',
 'score_differential',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'wind',
 'temp',
 'spread_line',
 'total_line']

In [13]:
vik.RBs

[RB:Kenneth Gainwell has 7 carries for 30 yards,
 RB:Will Shipley has 10 carries for 67 yards,
 RB:Saquon Barkley has 7 carries for 31 yards]

In [14]:
pbp.head(15)

,possession,quarter,down,ydstogo,goal_to_go,yardline_100,total_home_score,total_away_score,posteam_score,score_differential,wind,temp,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,spread_line,total_line,play_type,yards_gained
0,DET,1,1,10,0,35,0,0,0,0,1,87,900,1800,3600,-14.5,42.5,kickoff,0
1,DET,1,1,10,0,35,0,0,0,0,1,87,900,1800,3600,-14.5,42.5,run,10
2,DET,1,1,10,0,45,0,0,0,0,1,87,881,1781,3581,-14.5,42.5,pass,15
3,DET,1,1,10,0,60,0,0,0,0,1,87,851,1751,3551,-14.5,42.5,pass,0
4,DET,1,2,10,0,60,0,0,0,0,1,87,815,1715,3515,-14.5,42.5,run,12
5,DET,1,1,10,0,72,0,0,0,0,1,87,794,1694,3494,-14.5,42.5,run,-1
6,DET,1,2,11,0,71,0,0,0,0,1,87,767,1667,3467,-14.5,42.5,pass,0
7,DET,1,3,11,0,71,0,0,0,0,1,87,756,1656,3456,-14.5,42.5,pass,21
8,DET,1,1,10,1,92,0,0,0,0,1,87,720,1620,3420,-14.5,42.5,pass,0
9,DET,1,2,10,1,92,0,0,0,0,1,87,691,1591,3391,-14.5,42.5,run,8


In [15]:
pbp.loc[pbp.down==4]

,possession,quarter,down,ydstogo,goal_to_go,yardline_100,total_home_score,total_away_score,posteam_score,score_differential,wind,temp,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,spread_line,total_line,play_type,yards_gained
11,DET,1,4,2,1,100,0,0,0,0,1,87,622,1522,3322,-14.5,42.5,punt,0
14,PHI,1,4,3,0,32,0,0,0,0,1,87,567,1467,3267,-14.5,42.5,field_goal,0
17,DET,1,4,11,0,34,3,0,0,3,1,87,480,1380,3180,-14.5,42.5,field_goal,0
40,PHI,2,4,10,0,73,10,7,10,3,1,87,784,784,2584,-14.5,42.5,punt,0
45,DET,2,4,6,0,40,10,7,7,3,1,87,669,669,2469,-14.5,42.5,punt,0
48,PHI,2,4,10,0,27,10,7,10,3,1,87,593,593,2393,-14.5,42.5,field_goal,0
51,DET,2,4,6,0,77,10,7,7,3,1,87,513,513,2313,-14.5,42.5,punt,0
58,PHI,2,4,5,1,99,10,7,10,3,1,87,349,349,2149,-14.5,42.5,punt,0
68,DET,2,4,3,0,77,10,7,7,3,1,87,126,126,1926,-14.5,42.5,pass,0
73,PHI,3,4,5,0,41,10,7,10,3,1,87,900,1800,1800,-14.5,42.5,pass,0


In [16]:
game.pbp

[{'possession': 'DET',
  'quarter': 1,
  'down': 1,
  'ydstogo': 10,
  'goal_to_go': 0,
  'yardline_100': 35,
  'total_home_score': 0,
  'total_away_score': 0,
  'posteam_score': 0,
  'score_differential': 0,
  'wind': 1,
  'temp': 87,
  'quarter_seconds_remaining': 900,
  'half_seconds_remaining': 1800,
  'game_seconds_remaining': 3600,
  'spread_line': -14.5,
  'total_line': 42.5,
  'play_type': 'kickoff',
  'yards_gained': 0},
 {'possession': 'DET',
  'quarter': 1,
  'down': 1,
  'ydstogo': 10,
  'goal_to_go': 0,
  'yardline_100': 35,
  'total_home_score': 0,
  'total_away_score': 0,
  'posteam_score': 0,
  'score_differential': 0,
  'wind': 1,
  'temp': 87,
  'quarter_seconds_remaining': 900,
  'half_seconds_remaining': 1800,
  'game_seconds_remaining': 3600,
  'spread_line': -14.5,
  'total_line': 42.5,
  'play_type': 'run',
  'yards_gained': 10},
 {'possession': 'DET',
  'quarter': 1,
  'down': 1,
  'ydstogo': 10,
  'goal_to_go': 0,
  'yardline_100': 45,
  'total_home_score': 0,
